In [1]:
# %load ../loaders/imports.py
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import time
import pdb

sys.path.append('..')

from utils import gen_covariance, gen_beta2, gen_data, get_cov_list
from utils import selection_accuracy, estimation_error
from sklearn.linear_model import LassoLars, lasso_path, LinearRegression

from pyuoi.linear_model import UoI_Lasso
from pyuoi.linear_model import cassolasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [2]:
# %load ../loaders/datgen.py
n_features = 500
n_samples = 2000

sigma = gen_covariance(n_features, 0.25, 100, 1, 0)
beta = gen_beta2(n_features, n_features, sparsity = 0.2, betawidth = np.inf)
X, X_test, y, y_test, ss = gen_data(n_samples, n_features, kappa = 100, 
									covariance = sigma, beta = beta)

X = StandardScaler().fit_transform(X)
y -= np.mean(y)

In [3]:
uoi1 = UoI_Lasso()
uoi2 = cassolasso.UoI_Lasso()

In [4]:
# Time execution
# %time uoi1.fit(X, y.ravel())

In [5]:
# %time uoi2.fit(X, y.ravel())

In [6]:
# Assess similarity of results
random_state =  1234
uoi1 = UoI_Lasso(fit_intercept = False, random_state = random_state)
uoi2 = cassolasso.UoI_Lasso(fit_intercept = False, random_state = random_state)

uoi1.fit(X, y)
uoi2.fit(X, y)

KeyboardInterrupt: 

In [ ]:
selection_accuracy(beta.ravel(), uoi1.coef_)

In [ ]:
selection_accuracy(beta.ravel(), uoi2.coef_)

In [ ]:
estimation_error(beta.ravel(), uoi1.coef_)

In [ ]:
estimation_error(beta.ravel(), uoi2.coef_)

Test the equivalent modifications on UoI Elastic Net

In [ ]:
def GCV(X, y, l):
    n, n_features = X.shape
    H = X @ np.linalg.inv(X.T @ X + l * np.eye(n_features)) @ X.T
    mu = H @ y
    gcv = 1/n * np.linalg.norm(y.ravel() - mu.ravel())**2/(1 - 1/n * np.trace(H))**2
    return gcv

In [ ]:
# Examine the nature of the GCV ridge regularization function
l = np.linspace(0, 50, 500)
gcv = np.array([GCV(X, y, l_) for l_ in l])

In [ ]:
plt.plot(gcv)

In [ ]:
# Compare with what we get from RidgeCV
from sklearn.linear_model import RidgeCV, ElasticNetCV

In [ ]:
r = RidgeCV(alphas = np.linspace(0.01, 1000, 200), fit_intercept = False)
r.fit(X, y.ravel())

In [ ]:
r.alpha_

In [ ]:
np.linspace(0, 10, 100)[np.argmin(gcv)]

In [ ]:
# How about vs. ElasticNetCV?
e = ElasticNetCV(l1_ratio = [0.1, 0.5, 0.75, 0.9, 0.95, 0.99], fit_intercept = False, cv = 5)
e.fit(X, y.ravel())

In [ ]:
from sklearn.linear_model.coordinate_descent import _alpha_grid

In [ ]:
l1 = _alpha_grid(X, y, l1_ratio = 0.99)

In [ ]:
l2 = (1 - 0.1) * l1/2

In [ ]:
l2

In [4]:
from pyuoi.linear_model import UoI_ElasticNet
from pyuoi.linear_model import casso_en

In [5]:
uoi1 = casso_en.UoI_ElasticNet()

In [ ]:
%time uoi1.fit(X, y.ravel())